# 2 & 3 way Split and K-Folds
## ML Lab CSE-AIML (5th Sem) 2025

authors : <br>
[Mohamed Kamran 4NI23CI058](https://github.com/MohamedKamran)
<br>
[Akash 4NI24CI401](https://github.com/Akash-1305)

Dataset : https://www.kaggle.com/datasets/yasserh/titanic-dataset

# Load Data

In [1]:
import pandas as pd

df = pd.read_csv("/kaggle/input/titanic-dataset/Titanic-Dataset.csv")
df.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Training using Logistic Regression (2 Way Split)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = df[features]
y = df["Survived"]

X["Sex"] = LabelEncoder().fit_transform(X["Sex"])
X["Embarked"] = X["Embarked"].fillna("S")
X["Embarked"] = LabelEncoder().fit_transform(X["Embarked"])

imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/tmp/ipykernel_36/2216858733.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"] = LabelEncoder().fit_transform(X["Sex"])
/tmp/ipykernel_36/2216858733.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Embarked"] = X["Embarked"].fillna("S")   # fill missing with most common
/tmp/ipykernel_36/2216858733.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

Accuracy: 0.7910447761194029

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.84      0.83       165
           1       0.74      0.71      0.72       103

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268



In [17]:
new_passenger = pd.DataFrame([{
    "Pclass": 3,
    "Sex": "male",
    "Age": 25,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
}])

new_passenger["Sex"] = (new_passenger["Sex"] == "male").astype(int)
new_passenger["Embarked"] = new_passenger["Embarked"].map({'C':0,'Q':1,'S':2}) 

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
new_passenger_imputed = imputer.fit_transform(new_passenger)

prediction = model.predict(new_passenger_imputed)
probability = model.predict_proba(new_passenger_imputed)

print("Predicted Survival:", "Survived" if prediction[0]==1 else "Did Not Survive")

Predicted Survival: Did Not Survive


# K-Fold (5 Folds)

In [23]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import numpy as np

model = LogisticRegression(max_iter=1000)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, cv=kf, scoring="accuracy")

print("Accuracy for each fold:", scores)
print("Mean Accuracy:", np.mean(scores))

Accuracy for each fold: [0.81005587 0.79213483 0.84269663 0.7752809  0.78651685]
Mean Accuracy: 0.8013370158809867


# 3 Way Split (Train, Validate, Test)

In [25]:
# 0.2
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 0.8x0.75 = 0.6, 0.8x0.25 = 0.2
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print("Train samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])
print("Test samples:", X_test.shape[0])

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_val_pred = model.predict(X_val)
print("\nValidation Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nValidation Classification Report:\n", classification_report(y_val, y_val_pred))

y_test_pred = model.predict(X_test)
print("\nTest Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nTest Classification Report:\n", classification_report(y_test, y_test_pred))


Train samples: 534
Validation samples: 178
Test samples: 179

Validation Accuracy: 0.7921348314606742

Validation Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.85      0.84       110
           1       0.75      0.69      0.72        68

    accuracy                           0.79       178
   macro avg       0.78      0.77      0.78       178
weighted avg       0.79      0.79      0.79       178


Test Accuracy: 0.7877094972067039

Test Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.86      0.83       110
           1       0.75      0.67      0.71        69

    accuracy                           0.79       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.79      0.79      0.78       179



In [28]:
new_passenger = pd.DataFrame([{
    "Pclass": 3,
    "Sex": "female",
    "Age": 25,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S"
}])

new_passenger["Sex"] = (new_passenger["Sex"] == "male").astype(int)
new_passenger["Embarked"] = new_passenger["Embarked"].map({'C':0,'Q':1,'S':2}) 

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
new_passenger_imputed = imputer.fit_transform(new_passenger)

prediction = model.predict(new_passenger_imputed)
probability = model.predict_proba(new_passenger_imputed)

print("Predicted Survival:", "Survived" if prediction[0]==1 else "Did Not Survive")

Predicted Survival: Survived
